### Deps

In [169]:
!pip install pandas
!pip install python-docx

     -------------------------------------- 236.1/236.1 kB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 3.8/3.8 MB 804.9 kB/s eta 0:00:00
  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)


In [171]:
import re

import pandas as pd
from docx import Document

# Задание 1

### Load Data

In [3]:
xl_file = 'ranking_dataset_new.xlsx'
data = pd.read_excel(xl_file)
data

,id,author_name,title,issn,journal,year,address
0,WOS:000209908400008,"Tamosiunaite, Aurelija",Defining 'Lithuanian' Orthographic debates at ...,1387-6732,WRITTEN LANGUAGE AND LITERACY,2015,"Vytautas Magnus Univ, Fac Humanities, Dept Lit..."
1,WOS:000209919200006,"Zhang, Hui Andy",RECOGNITION THROUGH REINVENTION: THE MYTH OF C...,2013-3294,452 F-REVISTA DE TEORIA DE LA LITERATURA Y LIT...,2015,"Northwestern Univ, Evanston, IL 60208 USA."
2,WOS:000210161400001,"Bortnik, Boris",Interdisciplinary project-based learning: tech...,2156-7069,RESEARCH IN LEARNING TECHNOLOGY,2015,"Ural State Univ Econ, Ekaterinburg, Russia."
3,WOS:000210161400001,"Mironova, Ludmila",Interdisciplinary project-based learning: tech...,2156-7069,RESEARCH IN LEARNING TECHNOLOGY,2015,"Ural State Univ Econ, Ekaterinburg, Russia."
4,WOS:000210161400001,"Podshivalova, Ekaterina",Interdisciplinary project-based learning: tech...,2156-7069,RESEARCH IN LEARNING TECHNOLOGY,2015,"Ural State Univ Econ, Ekaterinburg, Russia."
...,...,...,...,...,...,...,...
21783,WOS:000589510700005,"Jaques, Susan","""This Heavy Thing"": Catherine the Great's Coro...",2051-1817,LUXURY-HISTORY CULTURE CONSUMPTION,2019,"Stanford Univ, Stanford, CA 94305 USA."
21784,WOS:000589783500009,"Mayofis, M.",FATHERS-HEROES AND DISCIPLINED SONS: CULTURAL ...,2219-3111,VESTNIK PERMSKOGO UNIVERSITETA-ISTORIYA-PERM U...,2019,"Natl Res Univ Higher Sch Econ, Myasnitskaya St..."
21785,WOS:000589783500014,"Janis-Isokangas, I.",FROM NATIONAL TO INTERNATIONAL - KONRAD ZILLIA...,2219-3111,VESTNIK PERMSKOGO UNIVERSITETA-ISTORIYA-PERM U...,2019,"Univ Helsinki, Aleksanteri Inst, PL 42, Helsin..."
21786,WOS:000590169500002,"Balkus, Muidaugas",Sociocultural Lituanization in Gymnasiums of N...,1392-0456,ISTORIJA,2019,"Vytautas Magnus Univ, Fac Humanities, Dept His..."


### 1. Из переменной address извлечь название организации и страну, добавив к датасету колонки org_name и country.

#### 1.1 Data investigation

In [121]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21788 entries, 0 to 21787
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           21788 non-null  object
 1   author_name  21788 non-null  object
 2   title        21788 non-null  object
 3   issn         21302 non-null  object
 4   journal      21788 non-null  object
 5   year         21788 non-null  int64 
 6   address      21788 non-null  object
 7   org_name     21328 non-null  object
 8   country      21328 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.5+ MB


In [5]:
data['address'].head(15).unique()

array(['Vytautas Magnus Univ, Fac Humanities, Dept Lithuanian Language, K Donelaicio G 52-205, LT-44244 Kaunas, Lithuania.',
       'Northwestern Univ, Evanston, IL 60208 USA.',
       'Ural State Univ Econ, Ekaterinburg, Russia.',
       'James Madison Univ, Intelligence Analysis, Harrisonburg, VA 22807 USA.',
       'Univ Kentucky, Patterson Sch Diplomacy & Int Commerce, Lexington, KY 40506 USA.',
       'Univ Kentucky, Dept Polit Sci, Lexington, KY 40506 USA.',
       'Charles Univ Prague, Fac Social Sci, Inst Int Studies, Prague, Czech Republic.',
       'Univ London, SOAS, Dept Anthropol & Sociol, London, England.',
       'Indiana Univ, Cent Eurasian Studies & Hist, Indianapolis, IN 46204 USA.',
       'Univ Kent, Ctr Crit Thought, Canterbury, Kent, England.',
       'Leiden Univ, Grotius Ctr Int Legal Studies, Leiden, Netherlands.'],
      dtype=object)

In [6]:
data['address'].describe()

count                              21788
unique                              9752
top       Kazan Fed Univ, Kazan, Russia.
freq                                1272
Name: address, dtype: object

In [7]:
# regex for university and country
org_regex = r"(.*?),"
country_regex = r"(\w*)\.$"

data['org_name'] = data['address'].str.extract(org_regex)
data['country'] = data['address'].str.extract(country_regex)
data[['org_name', 'country']].sample(5)

,org_name,country
16399,Wayne State Univ,USA
13,Univ Kent,England
15369,Lomonosov Moscow State Univ,Russia
192,Uppsala Univ,Sweden
8510,Georgetown Univ,USA


In [8]:
data.iloc[2398]

id                                           WOS:000380998000006
author_name                                  Keukeleire, Stephan
title          Voting Cohesion of the BRICS Countries in the ...
issn                                                   1075-2846
journal                                        GLOBAL GOVERNANCE
year                                                        2016
address                                   Could not be extracted
org_name                                                     NaN
country                                                      NaN
Name: 2398, dtype: object

•	количество публикаций в названии которых встречается Russia или Russian

In [70]:
data['title'].describe()

count                                                 21788
unique                                                11204
top       A hundred introductions to semiotics, for a mi...
freq                                                     19
Name: title, dtype: object

In [83]:
rus_regex = r"[Rr]ussia[n ]"

russian = data['title'].str.contains(rus_regex)
russian = data.loc[russian]

russian['title'].sample(1).iloc[0]

"Turning the Pushkin Museum into a 'Russian Tate': Informal creative\n\nlabour in a transitional cultural economy (the case of privately funded\n\nMoscow art centres)"

In [172]:
russian['title'].describe()

count                                                  4865
unique                                                 2550
top       The Factorial Structure of Spatial Abilities i...
freq                                                     13
Name: title, dtype: object

### 2. список ТОП-20 организаций по количеству авторов с указанием их количества

In [102]:
org_authors = data.groupby('org_name')['author_name'].nunique()
top_org_authors = org_authors.sort_values(ascending=False).head(20)
top_org_authors

org_name
Kazan Fed Univ                         2083
Natl Res Univ                           566
Natl Res Univ Higher Sch Econ           495
St Petersburg State Univ                484
Belgorod State Univ                     246
Lomonosov Moscow State Univ             238
Kazan Volga Reg Fed Univ                198
Stavropol State Agr Univ                180
Russian State Social Univ               164
Plekhanov Russian Univ Econ             159
Financial Univ Govt Russian Federat     144
Ural Fed Univ                           140
Peoples Friendship Univ Russia          126
North Eastern Fed Univ                  122
RUDN Univ                               121
Southern Fed Univ                       105
Univ Tartu                              103
Stavropol State Agrarian Univ            99
Moscow MV Lomonosov State Univ           86
Univ Helsinki                            86
Name: author_name, dtype: int64

In [108]:
data[data['org_name'] == 'Kazan Fed Univ']['author_name']

1572                     Salnikova, Alla
1911                 Bayanova, Larisa F.
1912                  Mustafin, Timur R.
2119                       Zolobov, Oleg
2418                Ankudinov, Andrei B.
                      ...               
21660                        Xu Jingting
21661     Chumakova, Galina Valentinovna
21662    Starostina, Olga Vyacheslavovna
21663       Alikberova, Alfiya Rafisovna
21664         Karamova, Yuliya Yurisovna
Name: author_name, Length: 3033, dtype: object

### 3. список ТОП-20 авторов по количеству публикаций с указанием их количества

In [103]:
title_authors = data.groupby('author_name')['title'].nunique()
top_author = title_authors.sort_values(ascending=False).head(20)
top_author_titles

author_name
Zamaletdinov, Radif R.            24
Kovalenko, Kseniya                23
Kovalenko, Kseniya E.             21
Zhatkin, Dmitry N.                21
Souleimanov, Emil Aslan           16
Gubareva, Anna                    16
Zaitseva, Natalia A.              16
Safiullin, Marat R.               14
Fedorov, Alexander                14
Larionova, Anna A.                14
Laruelle, Marlene                 13
Trukhachev, Vladimir Ivanovich    12
Bogoviz, Aleksei V.               12
Zamaletdinova, Gulnara F.         12
Rubtcova, Mariia                  12
Rokotyanskaya, Violetta V.        11
Makarychev, Andrey                11
Yashina, Tatiana A.               10
Elshin, Leonid A.                 10
Levitskaya, Anastasia             10
Name: title, dtype: int64

In [119]:
data[data['author_name'] == 'Zamaletdinov, Radif R.']['title'].sample(1).iloc[0]

'Realization of the Concepts Eget and Man in the Paramies of the Tatar\n\nand English Languages'

Соавторы случайной статьи Zamaletdinov, Radif R.

In [168]:
data[data['title'] == data[data['author_name'] == 'Zamaletdinov, Radif R.']['title'].sample(1).iloc[0]]['author_name']

14037    Gabdrakhmanova, Fanuza H.
14038         Jetibai, Janbolat M.
14039       Zamaletdinov, Radif R.
14040    Zamaletdinova, Gulnara F.
Name: author_name, dtype: object

### 4. список ТОП-20 организаций по количеству публикаций с указанием их количества

In [120]:
org_titles = data.groupby('org_name')['title'].nunique()
top_org_titles = org_titles.sort_values(ascending=False).head(20)
top_org_titles

org_name
Kazan Fed Univ                         1234
Natl Res Univ                           495
Natl Res Univ Higher Sch Econ           377
St Petersburg State Univ                376
Lomonosov Moscow State Univ             175
Plekhanov Russian Univ Econ             124
Ural Fed Univ                           111
Kazan Volga Reg Fed Univ                104
Russian State Social Univ               103
Univ Tartu                              102
Altai State Univ                         96
Financial Univ Govt Russian Federat      92
Univ Helsinki                            92
Univ Oxford                              90
North Eastern Fed Univ                   80
Peoples Friendship Univ Russia           78
Moscow MV Lomonosov State Univ           76
Univ Cambridge                           76
Belgorod State Univ                      76
Kings Coll London                        68
Name: title, dtype: int64

### 5. список ТОП-20 журналов  по количеству публикаций с указанием их количества

In [154]:
journal_titles = data.groupby('journal')['title'].nunique()
top_journal_titles = journal_titles.sort_values(ascending=False).head(20)
top_journal_titles

journal
MODERN JOURNAL OF LANGUAGE TEACHING METHODS                                                   374
RESEARCH JOURNAL OF PHARMACEUTICAL BIOLOGICAL AND CHEMICAL SCIENCES                           286
AMAZONIA INVESTIGA                                                                            250
TARIH KULTUR VE SANAT ARASTIRMALARI DERGISI-JOURNAL OF HISTORY CULTURE\n\nAND ART RESEARCH    234
AD ALTA-JOURNAL OF INTERDISCIPLINARY RESEARCH                                                 187
TURKISH ONLINE JOURNAL OF DESIGN ART AND COMMUNICATION                                        176
EUROPE-ASIA STUDIES                                                                           157
DILEMAS CONTEMPORANEOS-EDUCACION POLITICA Y VALORES                                           146
PSYCHOLOGY IN RUSSIA-STATE OF THE ART                                                         113
REVISTA SAN GREGORIO                                                                          102
RUSSIAN REVI

### 6. Save results

In [ ]:
document = Document()

table_1 = document.add_table()